# 相似度筛选

In [1]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,QuantileTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# 加载数据
df = pd.read_csv(r"F:\cache_data\frequency_filter\ky\ky_frequency_filter_less_feature_TL.csv")

In [24]:
# 删除缺失值
df = df.dropna(subset=['TZ'])

In [26]:
df.fillna(df.select_dtypes(include='number').mean(),inplace=True)

In [27]:
df

,TL,YL,TS,TZ,label,PRE,SRA,TMP,VAP,WIND,...,DEM,AnalyticalHillshading,DL,ASP_a,TWI3_a,TWI5_a,TPI3_a,TPI5_a,DZ,MRVBF_60
0,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,fish_net,92.416664,12677.500,15.166667,1.469167,1.700000,...,920.99240,1.253124,3,112.93275,4.501821,5.012647,100.021430,100.051380,5,0.326753
1,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,fish_net,92.416664,12677.500,15.166667,1.469167,1.700000,...,999.21160,1.423982,3,146.01454,3.568255,4.079081,100.019540,100.039780,5,0.010064
2,黄壤,黄泥土,黄泥土,死黄泥土,fish_net,92.416664,12677.500,15.166667,1.469167,1.700000,...,934.41064,1.022579,2,167.66680,5.704602,6.215428,100.000490,99.998900,5,0.574630
3,黄壤,黄泥土,黄泥土,死黄泥土,fish_net,92.416664,12677.500,15.166667,1.469167,1.700000,...,879.38776,1.048102,3,156.46916,4.435872,4.946697,100.067880,100.172700,5,0.528471
4,水稻土,潴育型,黄泥,黄泥,fish_net,92.666664,12655.333,14.883333,1.444167,1.733333,...,1105.48200,0.996057,7,149.77339,3.380327,3.891153,99.998405,99.991974,5,0.524177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32431,水稻土,淹育型,坡塝黄泥,黄砂泥,inner,92.083336,12574.500,14.600000,1.403333,1.808333,...,1184.11300,0.849817,7,233.49763,4.815844,5.326669,100.008800,100.010216,6,0.078464
32432,水稻土,淹育型,坡塝黄泥,黄砂泥,inner,95.083336,12565.667,13.566667,1.338333,1.916667,...,1337.10000,0.785191,9,-1.00000,8.006368,8.517193,100.000000,100.000000,5,4.988244
32433,水稻土,潴育型,黄泥,黄泥,inner,90.000000,12560.917,14.841667,1.432500,1.741667,...,914.23083,0.893939,1,163.19098,3.937913,4.448739,100.004810,100.013260,20,2.978450
32434,石灰土,黄色石灰土,黄色石灰土,薄层黄色石灰土,inner,90.166664,12640.083,15.783334,1.510000,1.608333,...,901.90564,0.497453,7,336.46426,3.151996,3.662821,100.109500,100.226616,5,0.935338


In [28]:
df.columns

Index(['TL', 'YL', 'TS', 'TZ', 'label', 'PRE', 'SRA', 'TMP', 'VAP', 'WIND',
       'BIO', 'Contrast', 'Correlation', 'Dissimilarity', 'Entropy',
       'Homogeneity', 'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment',
       'Variance', 'LON', 'LAT', 'Aspect', 'ChannelNetworkBaseLevel',
       'ChannelNetworkDistance', 'ClosedDepressions', 'ConvergenceIndex',
       'LSFactor', 'PlanCurvature', 'ProfileCurvature',
       'RelativeSlopePosition', 'Slope', 'TopographicPositionIndex',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'DEM',
       'AnalyticalHillshading', 'DL', 'ASP_a', 'TWI3_a', 'TWI5_a', 'TPI3_a',
       'TPI5_a', 'DZ', 'MRVBF_60'],
      dtype='object')

In [29]:
# 区分点位
no_calc_df = df[df['label']=='inner']
train_df = df[df['label']=='fish_net']

In [30]:
feature_columns = ['PRE', 'SRA', 'TMP', 'VAP', 'WIND',
       'BIO', 'Contrast', 'Correlation', 'Dissimilarity', 'Entropy',
       'Homogeneity', 'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment',
       'Variance', 'LON', 'LAT', 'Aspect', 'ChannelNetworkBaseLevel',
       'ChannelNetworkDistance', 'ConvergenceIndex',
       'LSFactor', 'PlanCurvature', 'ProfileCurvature',
       'RelativeSlopePosition', 'Slope', 'TopographicPositionIndex',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'DEM',
       'AnalyticalHillshading', 'DL', 'ASP_a', 'TWI3_a', 'TWI5_a', 'TPI3_a',
       'TPI5_a', 'DZ', 'MRVBF_60']

In [31]:
# 标签值
label_column = 'TZ'

In [32]:
# 获取标签的唯一值
label_values = train_df[label_column].unique()

In [33]:
# 获取标签的唯一值
label_values = train_df[label_column].unique()

# 初始化用于存储代表性样本的列表
representative_samples = []

# 设置相似度阈值
threshold = 0.9  # 根据需要调整

# 遍历每个标签值
for label_value in label_values:
    print(label_value)
    # 提取当前标签值的样本
    label_df = train_df[train_df[label_column] == label_value]
    
    # 检查样本数量
    num_samples = len(label_df)
    if num_samples == 1:
        # 如果样本数量为1，直接添加该样本
        representative_samples.append(label_df.iloc[0].values)
    else:
        # 对特征进行标准化
        scaler = StandardScaler()
        # scaler = MinMaxScaler()
        # scaler = MaxAbsScaler()
        # scaler = QuantileTransformer()
        features_scaled = scaler.fit_transform(label_df[feature_columns])

        # 计算样本间的余弦相似度
        similarity_matrix = cosine_similarity(features_scaled)

        # 初始化用于存储选定样本索引的集合
        selected_samples = set()

        # 遍历相似度矩阵，选择相似度高于阈值的样本
        for i in range(len(similarity_matrix)):
            for j in range(i + 1, len(similarity_matrix)):
                if similarity_matrix[i, j] > threshold:
                    selected_samples.add(i)
                    selected_samples.add(j)

        if len(selected_samples) == 0:
            print(label_value, f"共有样本{label_df.shape[0]}个","没有找到相似的样本")
            # 如果selected_samples为空，添加相似度最高的15%的样本
            num_to_add = math.ceil(0.75 * num_samples)
            if num_to_add > 0:
                top_indices = similarity_matrix.sum(axis=0).argsort()[::-1][:num_to_add]
                selected_samples.update(top_indices)

        # 从原始样本中提取选定的样本
        selected_samples_indices = label_df.index[list(selected_samples)]
        representative_samples.extend(df.loc[selected_samples_indices].values)

# 转换为 DataFrame 并显示部分结果
representative_samples_df = pd.DataFrame(representative_samples, columns=df.columns)


中层硅铁质黄壤
死黄泥土
黄泥
豆办泥土
黄泥土
薄层铁铝质黄壤
薄层黄色石灰土
薄层硅铁质黄壤
厚层铁铝质黄壤
厚层硅铁质黄壤
厚层次生黄色石灰土
厚层次生黄色石灰土 共有样本23个 没有找到相似的样本
小黄泥土
大土泥
中层次生黄色石灰土
中层次生黄色石灰土 共有样本34个 没有找到相似的样本
厚层黑色石灰土
厚层黑色石灰土 共有样本39个 没有找到相似的样本
中层铁铝质黄壤
深脚烂泥田
深脚烂泥田 共有样本29个 没有找到相似的样本
中层黄色石灰土
薄层黑色石灰土
浅脚烂泥田
浅脚烂泥田 共有样本119个 没有找到相似的样本
中层黑色石灰土
小土泥
小土泥 共有样本103个 没有找到相似的样本
大眼泥
大泥土
中层硅铝质黄壤
薄层硅铝质黄壤
中层硅质黄壤
黄胶泥
黄砂泥
黄砂土
厚层黄色石灰土
中层硅铁质黄壤性土
白云砂土
寡黄泥
寡黄泥 共有样本44个 没有找到相似的样本
薄层硅质黄壤
薄层硅质黄壤 共有样本56个 没有找到相似的样本
小黄泥
薄层淋溶黄色石灰土
死黄泥
死黄泥 共有样本51个 没有找到相似的样本
中层淋溶黄色石灰土
中层淋溶黄色石灰土 共有样本83个 没有找到相似的样本
 
潮砂土
薄层次生黄色石灰土
薄层次生黄色石灰土 共有样本4个 没有找到相似的样本
龙凤大眼泥
龙凤大眼泥 共有样本24个 没有找到相似的样本
灰泡黄泥土
灰泡黄泥土 共有样本85个 没有找到相似的样本
薄层硅铁质黄壤性土
薄层硅铁质黄壤性土 共有样本39个 没有找到相似的样本
扁砂土
黑色油砂石
黑色油砂石 共有样本45个 没有找到相似的样本
冷浸田
冷浸田 共有样本26个 没有找到相似的样本
火石砂土
厚层硅铝质黄壤
豆面泥土
豆面泥土 共有样本13个 没有找到相似的样本
苦鸭屎泥
苦鸭屎泥 共有样本13个 没有找到相似的样本
白砂土
白砂土 共有样本14个 没有找到相似的样本
豆办砂泥
豆办砂泥 共有样本26个 没有找到相似的样本
厚层淋溶黄色石灰土
厚层淋溶黄色石灰土 共有样本9个 没有找到相似的样本
冷砂田
冷砂田 共有样本19个 没有找到相似的样本
厚层硅铁质黄壤性土
厚层硅质黄壤
黄油砂泥
黄油砂泥 共有样本87个 没有找到相似的样本
潮砂泥
潮砂泥 共有样本27个 没有找到相似的样本
中层铁质黄壤
中层铁质黄壤 共有样本29个 没有找到相似的样本
潮泥
潮

In [34]:
len(representative_samples_df)

3074

In [35]:
# 检查类别是否丢失
old_type = train_df[label_column].value_counts()
new_type = representative_samples_df[label_column].value_counts()
old_type.shape,new_type.shape

((117,), (117,))

In [37]:
# 再合并
result_df = pd.concat([representative_samples_df,no_calc_df],ignore_index=True)
result_df = result_df.drop(columns=['label'])

In [38]:
result_df

,TL,YL,TS,TZ,PRE,SRA,TMP,VAP,WIND,BIO,...,DEM,AnalyticalHillshading,DL,ASP_a,TWI3_a,TWI5_a,TPI3_a,TPI5_a,DZ,MRVBF_60
0,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,89.583336,12615.833,15.991667,1.518333,1.633333,176.51971,...,967.97516,1.321254,3,105.042060,3.577492,4.088317,99.972084,99.924020,5,0.041386
1,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,93.500000,12602.083,14.008333,1.367500,1.866667,180.54616,...,1250.25450,0.562251,5,275.838230,5.302365,5.813191,100.007890,100.018290,5,0.557475
2,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,89.583336,12615.833,15.991667,1.518333,1.633333,176.51971,...,857.30190,1.415629,3,169.309630,3.828441,4.339267,100.004410,100.000320,5,0.002921
3,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,89.666664,12615.500,15.991667,1.519167,1.633333,176.57025,...,581.80280,1.568754,31,129.952870,3.740263,4.251089,99.971565,99.947970,5,0.000171
4,黄壤,黄壤,硅铁质黄壤,中层硅铁质黄壤,93.833336,12611.583,14.125000,1.372500,1.841667,181.41823,...,1245.12770,1.147106,3,100.531166,4.554472,5.065298,100.000630,99.999306,6,0.370417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12913,水稻土,淹育型,坡塝黄泥,黄砂泥,92.083336,12574.500,14.600000,1.403333,1.808333,179.39813,...,1184.11300,0.849817,7,233.497630,4.815844,5.326669,100.008800,100.010216,6,0.078464
12914,水稻土,淹育型,坡塝黄泥,黄砂泥,95.083336,12565.667,13.566667,1.338333,1.916667,182.47942,...,1337.10000,0.785191,9,-1.000000,8.006368,8.517193,100.000000,100.000000,5,4.988244
12915,水稻土,潴育型,黄泥,黄泥,90.000000,12560.917,14.841667,1.432500,1.741667,176.55756,...,914.23083,0.893939,1,163.190980,3.937913,4.448739,100.004810,100.013260,20,2.978450
12916,石灰土,黄色石灰土,黄色石灰土,薄层黄色石灰土,90.166664,12640.083,15.783334,1.510000,1.608333,177.27036,...,901.90564,0.497453,7,336.464260,3.151996,3.662821,100.109500,100.226616,5,0.935338


In [47]:
out_path = r"F:\cache_data\frequency_filter\ky\scaler_csv"
result_df.to_csv(os.path.join(out_path,'ky_stander_filter_all_type.csv'),index=False)
result_df.to_excel(os.path.join(out_path,'ky_stander_filter_all_type.xlsx'),index=False)

In [ ]:
import warnings
features = result_df.drop(columns=['TZ'])
# 禁用 FutureWarning 类型的警告
warnings.simplefilter(action='ignore', category=FutureWarning)

# 执行代码

# 恢复警告设置
# warnings.resetwarnings()

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 对特征进行标准化
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 应用 PCA
pca = PCA(n_components=30)
pca_result = pca.fit_transform(features_scaled)

# 应用 t-SNE，明确设置 init 为 'pca'
tsne = TSNE(n_components=2, random_state=0, init='pca', learning_rate='auto')  # 显式设置 init 和 learning_rate
tsne_result = tsne.fit_transform(features_scaled)

# 可视化 PCA 和 t-SNE 的结果
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)
plt.title('PCA Result')

plt.subplot(1, 2, 2)
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], alpha=0.5)
plt.title('t-SNE Result')

plt.show()

# 返回 PCA 和 t-SNE 结果以供进一步分析
pca_result, tsne_result


In [128]:
# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# 应用 PCA
pca = PCA(n_components=2)  # 使用两个主成分
X_pca = pca.fit_transform(X_scaled)

# 找到在第一主成分上具有最大和最小投影值的样本索引
first_pc = X_pca[:, 0]
representative_sample_indices = np.argpartition(first_pc, [0, -1])[:2]

# 选取具有代表性的样本
representative_samples = features.iloc[representative_sample_indices]

representative_samples

,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,ProfileCurvature,...,SecondMoment,Variance,PRE,SRA,TMP,VAP,WIND,BIO,LON,LAT
888,821.6,0.815088,3.894732,821.11770,0.48230,13.92083,-8.179223,27.389524,-0.018204,-0.019513,...,0.185185,4.666667,100.333336,13112.750,15.925000,1.568333,1.625000,192.72514,446545.0,2876000.0
1,870.8,1.131889,1.264116,835.30994,35.49005,13.92083,-2.361449,18.948938,-0.010550,-0.010593,...,0.185185,0.913580,101.916664,13154.333,15.766666,1.555833,1.658333,194.86105,443845.0,2861300.0


In [150]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity


# 应用 PCA
pca = PCA(n_components=2)  # 降至2维进行可视化
pca_result = pca.fit_transform(features)

# 计算样本在 PCA 降维后的空间中的坐标
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])

# 计算样本的密度
kde = KernelDensity(bandwidth=2)  # 根据需要调整 bandwidth
kde.fit(pca_result)

# 计算密度得分
density_scores = kde.score_samples(pca_result)

# 选择高密度样本
threshold = np.percentile(density_scores, 90)  # 根据需要调整阈值
high_density_samples = df[density_scores >= threshold]

# 显示高密度样本
print(high_density_samples)


      Unnamed: 0  TL  YL     TS            TZ     DEM  AnalyticalHillshading  \
0              0  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   961.5               0.706572   
1              2  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   870.8               1.131889   
6              8  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   869.8               0.820064   
8             10  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   847.1               0.733430   
16            21  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   854.1               1.295619   
...          ...  ..  ..    ...           ...     ...                    ...   
1826        4973  黄壤  黄壤  硅铝质黄壤  薄腐殖层厚土层硅铝质黄壤  1031.5               0.522569   
1897        5136  黄壤  黄壤  硅铝质黄壤  薄腐殖层中土层硅铝质黄壤   983.8               1.281300   
1902        5142  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   892.1               1.096558   
1963        5255  黄壤  黄壤  硅铝质黄壤  薄腐殖层薄土层硅铝质黄壤  1094.9               0.246648   
1997        5337  黄壤  黄壤  硅铁质黄壤  薄腐殖层厚土层硅铁质黄壤   838.9               0.785740   

        Aspect  ChannelNetworkBaseLevel

In [ ]:
# from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,QuantileTransformer
# label_column = 'TL'

# # 获取标签的唯一值
# label_values = df[label_column].unique()

# # 初始化用于存储代表性样本的列表
# representative_samples = []

# # 设置相似度阈值
# threshold = 0.9  # 根据需要调整

# # 遍历每个标签值
# for label_value in label_values:
#     # 提取当前标签值的样本
#     label_df = df[df[label_column] == label_value]
#     # 对特征进行标准化
#     scaler = StandardScaler()
#     # scaler = MinMaxScaler()
#     # scaler = MaxAbsScaler()
#     # scaler = QuantileTransformer()
#     features_scaled = scaler.fit_transform(label_df[feature_columns])

#     # 计算样本间的余弦相似度
#     similarity_matrix = cosine_similarity(features_scaled)
#     print(similarity_matrix)
#     # 初始化用于存储选定样本索引的集合
#     selected_samples = set()

#     # 遍历相似度矩阵，选择相似度高于阈值的样本
#     for i in range(len(similarity_matrix)):
#         for j in range(i + 1, len(similarity_matrix)):
#             if similarity_matrix[i, j] > threshold:
#                 selected_samples.add(i)
#                 selected_samples.add(j)

#     # 从原始样本中提取选定的样本
#     selected_samples_indices = label_df.index[list(selected_samples)]
#     representative_samples.extend(df.loc[selected_samples_indices].values)

# # 转换为 DataFrame 并显示部分结果
# representative_samples_df = pd.DataFrame(representative_samples, columns=df.columns)

